In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import json
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import DataLoader
from models import KGReasoning
from dataloader import TestDataset, TrainDataset, SingledirectionalOneShotIterator
from tensorboardX import SummaryWriter
import time
import pickle
from collections import defaultdict
from tqdm import tqdm
from util import flatten_query, list2tuple, parse_time, set_global_seed, eval_tuple

query_name_dict = {('e',('r',)): '1p', 
                    ('e', ('r', 'r')): '2p',
                    ('e', ('r', 'r', 'r')): '3p',
                    (('e', ('r',)), ('e', ('r',))): '2i',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r',))): '3i',
                    ((('e', ('r',)), ('e', ('r',))), ('r',)): 'ip',
                    (('e', ('r', 'r')), ('e', ('r',))): 'pi',
                    (('e', ('r',)), ('e', ('r', 'n'))): '2in',
                    (('e', ('r',)), ('e', ('r',)), ('e', ('r', 'n'))): '3in',
                    ((('e', ('r',)), ('e', ('r', 'n'))), ('r',)): 'inp',
                    (('e', ('r', 'r')), ('e', ('r', 'n'))): 'pin',
                    (('e', ('r', 'r', 'n')), ('e', ('r',))): 'pni',
                    (('e', ('r',)), ('e', ('r',)), ('u',)): '2u-DNF',
                    ((('e', ('r',)), ('e', ('r',)), ('u',)), ('r',)): 'up-DNF',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n',)): '2u-DM',
                    ((('e', ('r', 'n')), ('e', ('r', 'n'))), ('n', 'r')): 'up-DM'
                }
name_query_dict = {value: key for key, value in query_name_dict.items()}
all_tasks = list(name_query_dict.keys())

In [3]:
all_tasks

['1p',
 '2p',
 '3p',
 '2i',
 '3i',
 'ip',
 'pi',
 '2in',
 '3in',
 'inp',
 'pin',
 'pni',
 '2u-DNF',
 'up-DNF',
 '2u-DM',
 'up-DM']

In [4]:
def save_model(model, optimizer, save_variable_list, args):
    '''
    Save the parameters of the model and the optimizer,
    as well as some other variables such as step and learning_rate
    '''
    
    argparse_dict = vars(args)
    with open(os.path.join(args.save_path, 'config.json'), 'w') as fjson:
        json.dump(argparse_dict, fjson)

    torch.save({
        **save_variable_list,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()},
        os.path.join(args.save_path, 'checkpoint')
    )

def set_logger(args):
    '''
    Write logs to console and log file
    '''
    if args.do_train:
        log_file = os.path.join(args.save_path, 'train.log')
    else:
        log_file = os.path.join(args.save_path, 'test.log')

    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S',
        filename=log_file,
        filemode='a+'
    )
    if args.print_on_screen:
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
        console.setFormatter(formatter)
        logging.getLogger('').addHandler(console)

def log_metrics(mode, step, metrics):
    '''
    Print the evaluation logs
    '''
    for metric in metrics:
        logging.info('%s %s at step %d: %f' % (mode, metric, step, metrics[metric]))

def evaluate(model, tp_answers, fn_answers, args, dataloader, query_name_dict, mode, step, writer):
    '''
    Evaluate queries in dataloader
    '''
    average_metrics = defaultdict(float)
    all_metrics = defaultdict(float)

    metrics = model.test_step(model, tp_answers, fn_answers, args, dataloader, query_name_dict)
    num_query_structures = 0
    num_queries = 0
    for query_structure in metrics:
        log_metrics(mode+" "+query_name_dict[query_structure], step, metrics[query_structure])
        for metric in metrics[query_structure]:
            writer.add_scalar("_".join([mode, query_name_dict[query_structure], metric]), metrics[query_structure][metric], step)
            all_metrics["_".join([query_name_dict[query_structure], metric])] = metrics[query_structure][metric]
            if metric != 'num_queries':
                average_metrics[metric] += metrics[query_structure][metric]
        num_queries += metrics[query_structure]['num_queries']
        num_query_structures += 1

    for metric in average_metrics:
        average_metrics[metric] /= num_query_structures
        writer.add_scalar("_".join([mode, 'average', metric]), average_metrics[metric], step)
        all_metrics["_".join(["average", metric])] = average_metrics[metric]
    log_metrics('%s average'%mode, step, average_metrics)

    return all_metrics
        
def load_data(args, tasks):
    '''
    Load queries and remove queries not in tasks
    '''
    logging.info("loading data")
    train_queries = pickle.load(open(os.path.join(args.data_path, "train-queries.pkl"), 'rb'))
    train_answers = pickle.load(open(os.path.join(args.data_path, "train-answers.pkl"), 'rb'))
    valid_queries = pickle.load(open(os.path.join(args.data_path, "valid-queries.pkl"), 'rb'))
    valid_hard_answers = pickle.load(open(os.path.join(args.data_path, "valid-hard-answers.pkl"), 'rb'))
    valid_easy_answers = pickle.load(open(os.path.join(args.data_path, "valid-easy-answers.pkl"), 'rb'))
    test_queries = pickle.load(open(os.path.join(args.data_path, "test-queries.pkl"), 'rb'))
    test_hard_answers = pickle.load(open(os.path.join(args.data_path, "test-hard-answers.pkl"), 'rb'))
    test_easy_answers = pickle.load(open(os.path.join(args.data_path, "test-easy-answers.pkl"), 'rb'))
    
    # remove tasks not in args.tasks
    for name in all_tasks:
        if 'u' in name:
            name, evaluate_union = name.split('-')
        else:
            evaluate_union = args.evaluate_union
        if name not in tasks or evaluate_union != args.evaluate_union:
            query_structure = name_query_dict[name if 'u' not in name else '-'.join([name, evaluate_union])]
            if query_structure in train_queries:
                del train_queries[query_structure]
            if query_structure in valid_queries:
                del valid_queries[query_structure]
            if query_structure in test_queries:
                del test_queries[query_structure]

    return train_queries, train_answers, valid_queries, valid_hard_answers, valid_easy_answers, test_queries, test_hard_answers, test_easy_answers

In [5]:
class Args:
    cuda = True
    do_train = True
    do_valid = True
    do_test = True
    data_path = 'data/NELL-betae'
    negative_sample_size = 128
    hidden_dim = 400
    gamma = 60
    batch_size = 512
    test_batch_size = 1
    learning_rate = 0.0001
    cpu_num = 1
    save_path = None
    max_steps = 41
    warm_up_steps = None
    save_checkpoint_steps = 50000
    valid_steps = 15
    log_steps = 100
    test_log_steps = 10
    nentity = 0
    nrelation = 0
    geo = 'beta'
    #print_on_screen
    tasks = '1p.2p.3p.2i.3i.ip.pi.2in.3in.inp.pin.pni.2u.up'
    seed = 0
    beta_mode = "(1600,2)"
    prefix = None
    checkpoint_path = None
    evaluate_union = "DNF"
    print_on_screen = True
    box_mode = '(none,0.02'
    
    
args=Args()

In [6]:
    set_global_seed(args.seed)
    tasks = args.tasks.split('.')
    for task in tasks:
        if 'n' in task and args.geo in ['box', 'vec']:
            assert False, "Q2B and GQE cannot handle queries with negation"
    if args.evaluate_union == 'DM':
        assert args.geo == 'beta', "only BetaE supports modeling union using De Morgan's Laws"

    cur_time = parse_time()
    if args.prefix is None:
        prefix = 'logs'
    else:
        prefix = args.prefix
        
    

In [7]:
    print ("overwritting args.save_path")
    args.save_path = os.path.join(prefix, args.data_path.split('/')[-1], args.tasks, args.geo)
    if args.geo in ['box']:
        tmp_str = "g-{}-mode-{}".format(args.gamma, args.box_mode)
    elif args.geo in ['vec']:
        tmp_str = "g-{}".format(args.gamma)
    elif args.geo == 'beta':
        tmp_str = "g-{}-mode-{}".format(args.gamma, args.beta_mode)

    if args.checkpoint_path is not None:
        args.save_path = args.checkpoint_path
    else:
        args.save_path = os.path.join(args.save_path, tmp_str, cur_time.replace(":", "_"))

    if not os.path.exists(args.save_path):
        os.makedirs(args.save_path)

    print ("logging to", args.save_path)
    if not args.do_train: # if not training, then create tensorboard files in some tmp location
        writer = SummaryWriter('./logs-debug/unused-tb')
    else:
        writer = SummaryWriter(args.save_path)
    set_logger(args)

overwritting args.save_path
logging to logs\NELL-betae\1p.2p.3p.2i.3i.ip.pi.2in.3in.inp.pin.pni.2u.up\beta\g-60-mode-(1600,2)\2021.09.10-16_52_07


In [8]:
    with open('%s/stats.txt'%args.data_path) as f:
        entrel = f.readlines()
        nentity = int(entrel[0].split(' ')[-1])
        nrelation = int(entrel[1].split(' ')[-1])
    
    args.nentity = nentity
    args.nrelation = nrelation
    
    logging.info('-------------------------------'*3)
    logging.info('Geo: %s' % args.geo)
    logging.info('Data Path: %s' % args.data_path)
    logging.info('#entity: %d' % nentity)
    logging.info('#relation: %d' % nrelation)
    logging.info('#max steps: %d' % args.max_steps)
    logging.info('Evaluate unoins using: %s' % args.evaluate_union)

    train_queries, train_answers, valid_queries, valid_hard_answers, valid_easy_answers, test_queries, test_hard_answers, test_easy_answers = load_data(args, tasks)        

    logging.info("Training info:")

2021-09-10 16:53:44,080 INFO     ---------------------------------------------------------------------------------------------
2021-09-10 16:53:44,081 INFO     Geo: beta
2021-09-10 16:53:44,081 INFO     Data Path: data/NELL-betae
2021-09-10 16:53:44,082 INFO     #entity: 63361
2021-09-10 16:53:44,083 INFO     #relation: 400
2021-09-10 16:53:44,083 INFO     #max steps: 41
2021-09-10 16:53:44,084 INFO     Evaluate unoins using: DNF
2021-09-10 16:53:44,084 INFO     loading data
2021-09-10 16:53:55,923 INFO     Training info:


In [9]:
train_queries

defaultdict(set,
            {('e', ('r',)): {(1012, (281,)),
              (23400, (139,)),
              (35745, (176,)),
              (44072, (67,)),
              (10174, (93,)),
              (8965, (41,)),
              (35311, (34,)),
              (34006, (68,)),
              (49038, (2,)),
              (48642, (2,)),
              (11636, (160,)),
              (17705, (57,)),
              (18497, (133,)),
              (16775, (115,)),
              (34292, (18,)),
              (13252, (390,)),
              (26767, (69,)),
              (9667, (218,)),
              (19847, (54,)),
              (46643, (2,)),
              (20276, (33,)),
              (45740, (92,)),
              (8226, (28,)),
              (40879, (3,)),
              (6564, (53,)),
              (6168, (53,)),
              (14824, (123,)),
              (21083, (97,)),
              (20053, (12,)),
              (13779, (9,)),
              (50628, (204,)),
              (16174, (44,)),
         

In [31]:
    if args.do_train:
        for query_structure in train_queries:
            logging.info(query_name_dict[query_structure]+": "+str(len(train_queries[query_structure])))
        train_path_queries = defaultdict(set)
        train_other_queries = defaultdict(set)
        path_list = ['1p', '2p', '3p']
        for query_structure in train_queries:
            if query_name_dict[query_structure] in path_list:
                train_path_queries[query_structure] = train_queries[query_structure]
            else:
                train_other_queries[query_structure] = train_queries[query_structure]
        train_path_queries = flatten_query(train_path_queries)
        train_path_iterator = SingledirectionalOneShotIterator(DataLoader(
                                    TrainDataset(train_path_queries, nentity, nrelation, args.negative_sample_size, train_answers),
                                    batch_size=args.batch_size,
                                    shuffle=True,
                                    num_workers=args.cpu_num,
                                    collate_fn=TrainDataset.collate_fn
                                ))
        if len(train_other_queries) > 0:
            train_other_queries = flatten_query(train_other_queries)
            train_other_iterator = SingledirectionalOneShotIterator(DataLoader(
                                        TrainDataset(train_other_queries, nentity, nrelation, args.negative_sample_size, train_answers),
                                        batch_size=args.batch_size,
                                        shuffle=True,
                                        num_workers=args.cpu_num,
                                        collate_fn=TrainDataset.collate_fn
                                    ))
        else:
            train_other_iterator = None
    
    logging.info("Validation info:")
    if args.do_valid:
        for query_structure in valid_queries:
            logging.info(query_name_dict[query_structure]+": "+str(len(valid_queries[query_structure])))
        valid_queries = flatten_query(valid_queries)
        valid_dataloader = DataLoader(
            TestDataset(
                valid_queries, 
                args.nentity, 
                args.nrelation, 
            ), 
            batch_size=args.test_batch_size,
            num_workers=args.cpu_num, 
            collate_fn=TestDataset.collate_fn
        )


    logging.info("Test info:")
    if args.do_test:
        for query_structure in test_queries:
            logging.info(query_name_dict[query_structure]+": "+str(len(test_queries[query_structure])))
        test_queries = flatten_query(test_queries)
        test_dataloader = DataLoader(
            TestDataset(
                test_queries, 
                args.nentity, 
                args.nrelation, 
            ), 
            batch_size=args.test_batch_size,
            num_workers=args.cpu_num, 
            collate_fn=TestDataset.collate_fn
        )

    model = KGReasoning(
        nentity=nentity,
        nrelation=nrelation,
        hidden_dim=args.hidden_dim,
        gamma=args.gamma,
        geo=args.geo,
        use_cuda = args.cuda,
        box_mode=eval_tuple(args.box_mode),
        beta_mode = eval_tuple(args.beta_mode),
        test_batch_size=args.test_batch_size,
        query_name_dict = query_name_dict
    )

    logging.info('Model Parameter Configuration:')
    num_params = 0
    for name, param in model.named_parameters():
        logging.info('Parameter %s: %s, require_grad = %s' % (name, str(param.size()), str(param.requires_grad)))
        if param.requires_grad:
            num_params += np.prod(param.size())
    logging.info('Parameter Number: %d' % num_params)

    if args.cuda:
        model = model.cuda()
    
    if args.do_train:
        current_learning_rate = args.learning_rate
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, model.parameters()), 
            lr=current_learning_rate
        )
        warm_up_steps = args.max_steps // 2

    if args.checkpoint_path is not None:
        logging.info('Loading checkpoint %s...' % args.checkpoint_path)
        checkpoint = torch.load(os.path.join(args.checkpoint_path, 'checkpoint'))
        init_step = checkpoint['step']
        model.load_state_dict(checkpoint['model_state_dict'])

        if args.do_train:
            current_learning_rate = checkpoint['current_learning_rate']
            warm_up_steps = checkpoint['warm_up_steps']
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        logging.info('Ramdomly Initializing %s Model...' % args.geo)
        init_step = 0

    step = init_step 
    if args.geo == 'box':
        logging.info('box mode = %s' % args.box_mode)
    elif args.geo == 'beta':
        logging.info('beta mode = %s' % args.beta_mode)
    logging.info('tasks = %s' % args.tasks)
    logging.info('init_step = %d' % init_step)
    if args.do_train:
        logging.info('Start Training...')
        logging.info('learning_rate = %d' % current_learning_rate)
    logging.info('batch_size = %d' % args.batch_size)
    logging.info('hidden_dim = %d' % args.hidden_dim)
    logging.info('gamma = %f' % args.gamma)
    
    if args.do_train:
        training_logs = []
        # #Training Loop
        for step in range(init_step, args.max_steps):
            if step == 2*args.max_steps//3:
                args.valid_steps *= 4

            log = model.train_step(model, optimizer, train_path_iterator, args, step)
            for metric in log:
                writer.add_scalar('path_'+metric, log[metric], step)
            if train_other_iterator is not None:
                log = model.train_step(model, optimizer, train_other_iterator, args, step)
                for metric in log:
                    writer.add_scalar('other_'+metric, log[metric], step)
                log = model.train_step(model, optimizer, train_path_iterator, args, step)

            training_logs.append(log)

            if step >= warm_up_steps:
                current_learning_rate = current_learning_rate / 5
                logging.info('Change learning_rate to %f at step %d' % (current_learning_rate, step))
                optimizer = torch.optim.Adam(
                    filter(lambda p: p.requires_grad, model.parameters()), 
                    lr=current_learning_rate
                )
                warm_up_steps = warm_up_steps * 1.5
            
            if step % args.save_checkpoint_steps == 0:
                save_variable_list = {
                    'step': step, 
                    'current_learning_rate': current_learning_rate,
                    'warm_up_steps': warm_up_steps
                }
                save_model(model, optimizer, save_variable_list, args)

            if step % args.valid_steps == 0 and step > 0:
                if args.do_valid:
                    logging.info('Evaluating on Valid Dataset...')
                    valid_all_metrics = evaluate(model, valid_easy_answers, valid_hard_answers, args, valid_dataloader, query_name_dict, 'Valid', step, writer)

                if args.do_test:
                    logging.info('Evaluating on Test Dataset...')
                    test_all_metrics = evaluate(model, test_easy_answers, test_hard_answers, args, test_dataloader, query_name_dict, 'Test', step, writer)
                
            if step % args.log_steps == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    metrics[metric] = sum([log[metric] for log in training_logs])/len(training_logs)

                log_metrics('Training average', step, metrics)
                training_logs = []

        save_variable_list = {
            'step': step, 
            'current_learning_rate': current_learning_rate,
            'warm_up_steps': warm_up_steps
        }
        save_model(model, optimizer, save_variable_list, args)
        
    try:
        print (step)
    except:
        step = 0

    if args.do_test:
        logging.info('Evaluating on Test Dataset...')
        test_all_metrics = evaluate(model, test_easy_answers, test_hard_answers, args, test_dataloader, query_name_dict, 'Test', step, writer)

    logging.info("Training finished!!")

2021-09-10 16:42:48,334 INFO     ---------------------------------------------------------------------------------------------
2021-09-10 16:42:48,334 INFO     ---------------------------------------------------------------------------------------------
2021-09-10 16:42:48,334 INFO     ---------------------------------------------------------------------------------------------
2021-09-10 16:42:48,334 INFO     ---------------------------------------------------------------------------------------------
2021-09-10 16:42:48,337 INFO     Geo: beta
2021-09-10 16:42:48,337 INFO     Geo: beta
2021-09-10 16:42:48,337 INFO     Geo: beta
2021-09-10 16:42:48,337 INFO     Geo: beta
2021-09-10 16:42:48,339 INFO     Data Path: data/NELL-betae
2021-09-10 16:42:48,339 INFO     Data Path: data/NELL-betae
2021-09-10 16:42:48,339 INFO     Data Path: data/NELL-betae
2021-09-10 16:42:48,339 INFO     Data Path: data/NELL-betae
2021-09-10 16:42:48,341 INFO     #entity: 63361
2021-09-10 16:42:48,341 INFO    

overwritting args.save_path
logging to logs\NELL-betae\1p.2p.3p.2i.3i.ip.pi.2in.3in.inp.pin.pni.2u.up\beta\g-60-mode-(1600,2)\2021.09.10-16_42_48


2021-09-10 16:43:04,148 INFO     Training info:
2021-09-10 16:43:04,148 INFO     Training info:
2021-09-10 16:43:04,148 INFO     Training info:
2021-09-10 16:43:04,148 INFO     Training info:
2021-09-10 16:43:04,150 INFO     1p: 107982
2021-09-10 16:43:04,150 INFO     1p: 107982
2021-09-10 16:43:04,150 INFO     1p: 107982
2021-09-10 16:43:04,150 INFO     1p: 107982
2021-09-10 16:43:04,152 INFO     2p: 107982
2021-09-10 16:43:04,152 INFO     2p: 107982
2021-09-10 16:43:04,152 INFO     2p: 107982
2021-09-10 16:43:04,152 INFO     2p: 107982
2021-09-10 16:43:04,154 INFO     3p: 107982
2021-09-10 16:43:04,154 INFO     3p: 107982
2021-09-10 16:43:04,154 INFO     3p: 107982
2021-09-10 16:43:04,154 INFO     3p: 107982
2021-09-10 16:43:04,156 INFO     2i: 107982
2021-09-10 16:43:04,156 INFO     2i: 107982
2021-09-10 16:43:04,156 INFO     2i: 107982
2021-09-10 16:43:04,156 INFO     2i: 107982
2021-09-10 16:43:04,158 INFO     3i: 107982
2021-09-10 16:43:04,158 INFO     3i: 107982
2021-09-10 16:43

2021-09-10 16:43:07,715 INFO     Parameter entity_embedding: torch.Size([63361, 800]), require_grad = True
2021-09-10 16:43:07,715 INFO     Parameter entity_embedding: torch.Size([63361, 800]), require_grad = True
2021-09-10 16:43:07,715 INFO     Parameter entity_embedding: torch.Size([63361, 800]), require_grad = True
2021-09-10 16:43:07,717 INFO     Parameter relation_embedding: torch.Size([400, 400]), require_grad = True
2021-09-10 16:43:07,717 INFO     Parameter relation_embedding: torch.Size([400, 400]), require_grad = True
2021-09-10 16:43:07,717 INFO     Parameter relation_embedding: torch.Size([400, 400]), require_grad = True
2021-09-10 16:43:07,717 INFO     Parameter relation_embedding: torch.Size([400, 400]), require_grad = True
2021-09-10 16:43:07,719 INFO     Parameter center_net.layer1.weight: torch.Size([800, 800]), require_grad = True
2021-09-10 16:43:07,719 INFO     Parameter center_net.layer1.weight: torch.Size([800, 800]), require_grad = True
2021-09-10 16:43:07,719 I

2021-09-10 16:43:53,844 INFO     Training average loss at step 0: 11.965556
2021-09-10 16:43:53,844 INFO     Training average loss at step 0: 11.965556
2021-09-10 16:43:53,844 INFO     Training average loss at step 0: 11.965556
2021-09-10 16:43:53,844 INFO     Training average loss at step 0: 11.965556
2021-09-10 16:44:01,209 INFO     Evaluating on Valid Dataset...
2021-09-10 16:44:01,209 INFO     Evaluating on Valid Dataset...
2021-09-10 16:44:01,209 INFO     Evaluating on Valid Dataset...
2021-09-10 16:44:01,209 INFO     Evaluating on Valid Dataset...
  0%|                                                                                        | 0/68910 [00:00<?, ?it/s]2021-09-10 16:44:06,971 INFO     Evaluating the model... (0/68910)
2021-09-10 16:44:06,971 INFO     Evaluating the model... (0/68910)
2021-09-10 16:44:06,971 INFO     Evaluating the model... (0/68910)
2021-09-10 16:44:06,971 INFO     Evaluating the model... (0/68910)
  0%|                                                

...

  4%|███▏                                                                         | 2867/68910 [01:16<26:55, 40.89it/s]2021-09-10 16:45:17,824 INFO     Evaluating the model... (2870/68910)
2021-09-10 16:45:17,824 INFO     Evaluating the model... (2870/68910)
2021-09-10 16:45:17,824 INFO     Evaluating the model... (2870/68910)
2021-09-10 16:45:17,824 INFO     Evaluating the model... (2870/68910)
  4%|███▏                                                                         | 2877/68910 [01:16<26:51, 40.99it/s]2021-09-10 16:45:18,068 INFO     Evaluating the model... (2880/68910)
2021-09-10 16:45:18,068 INFO     Evaluating the model... (2880/68910)
2021-09-10 16:45:18,068 INFO     Evaluating the model... (2880/68910)
2021-09-10 16:45:18,068 INFO     Evaluating the model... (2880/68910)
  4%|███▏                                                                         | 2887/68910 [01:16<26:49, 41.03it/s]2021-09-10 16:45:18,312 INFO     Evaluating the model... (2890/68910)
2021-09-10 16

2021-09-10 16:45:22,769 INFO     Evaluating the model... (3070/68910)
2021-09-10 16:45:22,769 INFO     Evaluating the model... (3070/68910)
  4%|███▍                                                                         | 3076/68910 [01:21<27:15, 40.25it/s]2021-09-10 16:45:23,015 INFO     Evaluating the model... (3080/68910)
2021-09-10 16:45:23,015 INFO     Evaluating the model... (3080/68910)
2021-09-10 16:45:23,015 INFO     Evaluating the model... (3080/68910)
2021-09-10 16:45:23,015 INFO     Evaluating the model... (3080/68910)
  4%|███▍                                                                         | 3090/68910 [01:22<28:01, 39.14it/s]2021-09-10 16:45:23,283 INFO     Evaluating the model... (3090/68910)
2021-09-10 16:45:23,283 INFO     Evaluating the model... (3090/68910)
2021-09-10 16:45:23,283 INFO     Evaluating the model... (3090/68910)
2021-09-10 16:45:23,283 INFO     Evaluating the model... (3090/68910)
  4%|███▍                                                     

  5%|███▋                                                                         | 3279/68910 [01:26<27:39, 39.54it/s]2021-09-10 16:45:27,998 INFO     Evaluating the model... (3280/68910)
2021-09-10 16:45:27,998 INFO     Evaluating the model... (3280/68910)
2021-09-10 16:45:27,998 INFO     Evaluating the model... (3280/68910)
2021-09-10 16:45:27,998 INFO     Evaluating the model... (3280/68910)
  5%|███▋                                                                         | 3288/68910 [01:26<27:27, 39.84it/s]2021-09-10 16:45:28,244 INFO     Evaluating the model... (3290/68910)
2021-09-10 16:45:28,244 INFO     Evaluating the model... (3290/68910)
2021-09-10 16:45:28,244 INFO     Evaluating the model... (3290/68910)
2021-09-10 16:45:28,244 INFO     Evaluating the model... (3290/68910)
  5%|███▋                                                                         | 3300/68910 [01:27<27:49, 39.31it/s]2021-09-10 16:45:28,505 INFO     Evaluating the model... (3300/68910)
2021-09-10 16

2021-09-10 16:45:33,120 INFO     Evaluating the model... (3480/68910)
2021-09-10 16:45:33,120 INFO     Evaluating the model... (3480/68910)
  5%|███▉                                                                         | 3488/68910 [01:32<27:05, 40.24it/s]2021-09-10 16:45:33,365 INFO     Evaluating the model... (3490/68910)
2021-09-10 16:45:33,365 INFO     Evaluating the model... (3490/68910)
2021-09-10 16:45:33,365 INFO     Evaluating the model... (3490/68910)
2021-09-10 16:45:33,365 INFO     Evaluating the model... (3490/68910)
  5%|███▉                                                                         | 3498/68910 [01:32<26:53, 40.54it/s]2021-09-10 16:45:33,614 INFO     Evaluating the model... (3500/68910)
2021-09-10 16:45:33,614 INFO     Evaluating the model... (3500/68910)
2021-09-10 16:45:33,614 INFO     Evaluating the model... (3500/68910)
2021-09-10 16:45:33,614 INFO     Evaluating the model... (3500/68910)
  5%|███▉                                                     

KeyboardInterrupt: 